# aoc day 7

<img src="./images/day7.jpeg" alt="elf looking surprised at terminal output" width="200"/>

> prompt: *elf looking surprised at terminal output*

In [2]:
day = 7

In [3]:
%load_ext dotenv
%load_ext autoreload
%load_ext jupyter_black

In [4]:
import os
import sys
import math

sys.path.append(os.path.join(*os.path.split(os.getcwd())[0:-1]))
from scripts import aoc
from IPython.display import display, HTML

%autoreload 2
%dotenv

In [231]:
# problem_input = """$ cd /
# $ ls
# dir a
# 14848514 b.txt
# 8504156 c.dat
# dir d
# $ cd a
# $ ls
# dir e
# 29116 f
# 2557 g
# 62596 h.lst
# $ cd e
# $ ls
# 584 i
# $ cd ..
# $ cd ..
# $ cd d
# $ ls
# 4060174 j
# 8033020 d.log
# 5626152 d.ext
# 7214296 k
# """

problem_input = aoc.http_get_aoc_day(2022, day)

## preprocessing

follow the `cd` commands, appending that to an array called `path`. if the line isn't a command, we append the current path to the file-tree while keeping a `__contents` sum of all the files in that folder.

### in summary:

**for line**

    - if line starts with $
        - pop if the directory is ..
        - push if the command is cd
        **continue**

    - split on whitespace
        - if size = dir, **continue**
        - cast size to int
            - keep track of tree_item as the current node
            - walk through the path, defining the nodes if need be
            - if this is the end of the path, add the current file's size
        
this isn't incredibly fast but it is intuitive to me

In [232]:
lines = problem_input.split("\n")
lines.pop()

path = []
file_tree = {}
for line in lines:
    if line.startswith("$"):
        command = line[2:4]
        payload = line[5:]
        if command == "cd":
            if payload == "..":
                path.pop()
            else:
                path.append(payload)
        continue

    (size, name) = line.split(" ")

    if size == "dir":
        continue

    size = int(size)
    tree_item = file_tree
    for i, dir_name in enumerate(path):
        tree_item = tree_item.setdefault(dir_name, {})

        if i == len(path) - 1:
            tree_item["__contents"] = tree_item.get("__contents", 0) + size

In [234]:
print(json.dumps(file_tree, sort_keys=True, indent=4))

{
    "/": {
        "__contents": 23352670,
        "a": {
            "__contents": 94269,
            "e": {
                "__contents": 584
            }
        },
        "d": {
            "__contents": 24933642
        }
    }
}


### aggregating directory size

now recursively walk through the tree. counting the size of the children.

In [ ]:
def compute_file_tree_bounds():
    results = []

    def handle_folder(name, folder):
        contents = folder.get("__contents", 0)

        for key in folder:
            value = folder[key]

            if isinstance(value, dict):
                size = handle_folder(key, value)
                contents = contents + size

        results.append((name, contents))
        return contents

    handle_folder("/", file_tree["/"])
    return results


computed_dirs = compute_file_tree_bounds()

In [236]:
print(computed_dirs)

[('e', 584), ('a', 94853), ('d', 24933642), ('/', 48381165)]


## part one

In [188]:
output = sum(map(lambda x: x[1], filter(lambda x: x[1] <= 100000, computed_dirs)))

95437


In [115]:
display(HTML(aoc.http_post_answer(2022, day, 1, output)))

## part two

In [214]:
root_dir = next(dir[1] for dir in computed_dirs if dir[0] == "/")

required_size = 70000000 - root_dir

output = sorted(
    list(
        filter(
            lambda x: x[1] + required_size >= 30000000,
            computed_dirs,
        )
    ),
    key=lambda x: x[1],
)[0][1]
print(output)

2086088


In [215]:
display(HTML(aoc.http_post_answer(2022, day, 2, output)))

## retrospective

i didn't like this one. it took me like, an hour and a half.